In [1]:
# numpy downgrade가 필요할 수 있음
!python -m pip install numpy==1.2

In [ ]:
!pip install ensemble_boxes

In [5]:
import pandas as pd
from ensemble_boxes import *
import numpy as np
from pycocotools.coco import COCO

ImportError: Numba needs NumPy 1.20 or less

In [3]:
# ensemble csv files
submission_files = ['./output1.csv', './output2.csv'] # submission lists
submission_df = [pd.read_csv(file) for file in submission_files]

In [4]:
image_ids = submission_df[0]['image_id'].tolist()

In [5]:
annotation = '../dataset/test.json'
coco = COCO(annotation)

loading annotations into memory...
Done (t=0.01s)
creating index...
index created!


In [6]:
prediction_strings = []
file_names = []
iou_thr = 0.4

for i, image_id in enumerate(image_ids):
    prediction_string = ''
    boxes_list = []
    scores_list = []
    labels_list = []
    image_info = coco.loadImgs(i)[0]
    
    for df in submission_df:
        predict_string = df[df['image_id'] == image_id]['PredictionString'].tolist()[0]
        predict_list = str(predict_string).split()
        
        if len(predict_list)==0 or len(predict_list)==1:
            continue
            
        predict_list = np.reshape(predict_list, (-1, 6))
        box_list = []
        
        for box in predict_list[:, 2:6].tolist():
            box[0] = float(box[0]) / image_info['width']
            box[1] = float(box[1]) / image_info['height']
            box[2] = float(box[2]) / image_info['width']
            box[3] = float(box[3]) / image_info['height']
            box_list.append(box)
            
        boxes_list.append(box_list)
        scores_list.append(list(map(float, predict_list[:, 1].tolist())))
        labels_list.append(list(map(int, predict_list[:, 0].tolist())))
    
    if len(boxes_list):
        boxes, scores, labels = nms(boxes_list, scores_list, labels_list, iou_thr=iou_thr)
        for box, score, label in zip(boxes, scores, labels):
            prediction_string += str(label) + ' ' + str(score) + ' ' + str(box[0] * image_info['width']) + ' ' + str(box[1] * image_info['height']) + ' ' + str(box[2] * image_info['width']) + ' ' + str(box[3] * image_info['height']) + ' '
    
    prediction_strings.append(prediction_string)
    file_names.append(image_id)

In [7]:
submission = pd.DataFrame()
submission['PredictionString'] = prediction_strings
submission['image_id'] = file_names
submission.to_csv('submission_ensemble.csv')

submission.head()

,PredictionString,image_id
0,0 1.0 391.1 408.4 657.3 601.4 0 0.201180130243...,test/0000.jpg
1,0 0.203600212931633 684.6026611328125 642.4695...,test/0001.jpg
2,0 0.34274938702583313 306.0867919921875 359.00...,test/0002.jpg
3,0 0.06809787452220917 0.0 1.2754687070846558 1...,test/0003.jpg
4,0 1.0 66.0 502.3 1022.6 1023.7 0 1.0 653.9 275...,test/0004.jpg
